In [1]:
from oriented_detr.models.registry import MODULE_BUILD_FUNCS
assert 'odetr' in MODULE_BUILD_FUNCS._module_dict
build_func = MODULE_BUILD_FUNCS.get('odetr')

/home/vadim/miniconda3/envs/aerial_detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vadim/miniconda3/envs/aerial_detection/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [20]:
import argparse
from oriented_detr.main import get_args_parser

import shlex
argString = '-c oriented_detr/config/dota/4scale_swint_ss_dota.py'

parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args(shlex.split(argString))

In [21]:
from oriented_detr.util.slconfig import DictAction, SLConfig
import json
import torch
import os

# load cfg file and update the args
print("Loading config file from {}".format(args.config_file))
cfg = SLConfig.fromfile(args.config_file)
if args.options is not None:
    cfg.merge_from_dict(args.options)
# if args.rank == 0:
#     save_cfg_path = os.path.join(args.output_dir, "config_cfg.py")
#     cfg.dump(save_cfg_path)
#     save_json_path = os.path.join(args.output_dir, "config_args_raw.json")
#     with open(save_json_path, 'w') as f:
#         json.dump(vars(args), f, indent=2)
cfg_dict = cfg._cfg_dict.to_dict()
args_vars = vars(args)
for k,v in cfg_dict.items():
    if k not in args_vars:
        setattr(args, k, v)
        print(k, v)
    else:
        raise ValueError("Key {} can used by args only".format(k))

# update some new args temporally
if not getattr(args, 'use_ema', None):
    args.use_ema = False
if not getattr(args, 'debug', None):
    args.debug = False


if args.frozen_weights is not None:
    assert args.masks, "Frozen training is meant for segmentation only"
print(args)

device = torch.device(args.device)


model, criterion, postprocessors = build_func(args)

Loading config file from oriented_detr/config/dota/4scale_res50_ss_dota.py
num_classes 91
ignore_cls [10, 12]
inf_conf 0.005
lr 0.0001
param_dict_type default
lr_backbone 1e-05
lr_backbone_names ['backbone.0']
lr_linear_proj_names ['reference_points', 'sampling_offsets']
lr_linear_proj_mult 0.1
ddetr_lr_param False
batch_size 2
weight_decay 0.0001
epochs 36
save_checkpoint_interval 1
clip_max_norm 0.1
onecyclelr False
multi_step_lr True
lr_drop_list [27, 33]
modelname odetr
frozen_weights None
backbone resnet50
use_checkpoint False
dilation False
position_embedding sine
pe_temperatureH 20
pe_temperatureW 20
return_interm_indices [1, 2, 3]
backbone_freeze_keywords None
enc_layers 6
dec_layers 6
unic_layers 0
pre_norm False
dim_feedforward 2048
hidden_dim 256
dropout 0.0
nheads 8
num_queries 900
num_points 13
query_dim 4
num_patterns 0
pdetr3_bbox_embed_diff_each_layer False
pdetr3_refHW -1
random_refpoints_xy False
fix_refpoints_hw -1
dabdetr_yolo_like_anchor_update False
dabdetr_deform

/home/vadim/miniconda3/envs/aerial_detection/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vadim/miniconda3/envs/aerial_detection/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
sum(p.numel() for p in model.parameters())

57689842

In [4]:
import cv2
import numpy as np
import torchvision.transforms.functional as F
import torch

img = cv2.imread("DOTAv2/images/P0000.png")
img_size = 1024
img_h, img_w = img.shape[:2]
img_new = np.zeros((img_size, img_size, 3), dtype=np.uint8)
length_max = max([img_h, img_w])
img_w_new = int(img_size / length_max * img_w)
img_h_new = int(img_size / length_max * img_h)
img = cv2.resize(img, (img_w_new, img_h_new))
img[0:img_h_new, 0:img_w_new, :] = img[:, :, :]

img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).float() / 255
img = F.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [5]:
img.unsqueeze(0).shape

torch.Size([1, 3, 1024, 721])

In [22]:
checkpoint = torch.load('weights/swin_t_dota_ss.pth')

/tmp/ipykernel_21816/1210984230.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('weights/r50_dota_ss.pth')


In [23]:
model.eval()

model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [25]:
model = model.cuda()


model(img.unsqueeze(0).cuda())

/home/vadim/miniconda3/envs/aerial_detection/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 3.80 GiB of which 8.44 MiB is free. Including non-PyTorch memory, this process has 3.78 GiB memory in use. Of the allocated memory 3.62 GiB is allocated by PyTorch, and 60.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)